In [10]:
%load_ext autoreload
%autoreload 2

from pytorch_models_imp.resnet import ResNet, resnet152
from pytorch_models_imp.mobilenet_v2 import InvertedResidual, MobileNetV2, mobile_net_config
from pytorch_models_imp.mobilenet_v3 import InvertedResidualV3, mobilenet_v3_small, mobilenet_v3_large
from pytorch_models_imp.deeplabv3 import DeepLabv3
from torchvision.models.resnet import resnet50
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
def calculate_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

### Input example

In [12]:
x = torch.randn(2, 3, 224, 224)
n_x, c_x, h_x, w_x = x.shape

w = 112
h = 112
b = 2
in_ch = 16
for_inverted = torch.randn(b, in_ch, h, w)

## Resnet

In [13]:
num_blocks = [3, 4, 6, 3] # resnet50
resnet = ResNet(num_blocks=num_blocks, in_channels=3, num_classes=1000, dilation=2)
out = resnet(x)

resnet_pytorch = resnet50(replace_stride_with_dilation=[False, False, True])
assert out.shape == (2, 1000)
print(f"Number of parameters in resnet: {calculate_params(resnet)}")
print(f"Number of parameters in resnet pytorch: {calculate_params(resnet_pytorch)}")
print(f"Ratio: {calculate_params(resnet) / calculate_params(resnet_pytorch)}")

/home/evstifeev/anaconda3/envs/nn/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Number of parameters in resnet: 25583592
Number of parameters in resnet pytorch: 25557032
Ratio: 1.0010392443066158


## Mobilenet v2

In [14]:
# inverted residual
out_ch = 24
inverted_residual = InvertedResidual(in_ch, out_ch, expansion=6, stride=2)
assert inverted_residual(for_inverted).shape == (2, out_ch, w // 2, h // 2)

In [15]:
mobile_net_config # t, c, n, s

[[1, 16, 1, 1],
 [6, 24, 2, 2],
 [6, 32, 3, 2],
 [6, 64, 4, 2],
 [6, 96, 3, 1],
 [6, 160, 3, 2],
 [6, 320, 1, 1]]

In [16]:
# mobilenet_v2
mobile_net = MobileNetV2(mobile_net_config, in_channels=3, num_classes=1000)
out = mobile_net(x)
assert out.shape == (2, 1000)
print(f"Number of parameters in mobilenet v2: {calculate_params(mobile_net)}")

Number of parameters in mobilenet v2: 3521928


## Mobilenet v3

In [17]:
inverted_residuals = InvertedResidualV3(in_ch, out_ch=24, activation='SH', kernel_size=3, inner_ch=64, use_se=True, stride=2)
assert inverted_residuals(for_inverted).shape == (2, 24, 56, 56)

### Small mobilenet

In [18]:
small_mobile_net = mobilenet_v3_small(in_channels=3, num_classes=1000)
out = small_mobile_net(x)
assert out.shape == (2, 1000)

In [19]:
from torchvision.models.mobilenetv3 import mobilenet_v3_small as pytorch_mobilenet_v3_small
mobile_net_pytorch = pytorch_mobilenet_v3_small()
params_in_small_mobile_net_v3 = calculate_params(small_mobile_net)
params_in_small_mobile_net_v3_pytorch = calculate_params(mobile_net_pytorch)
print(f"Number of parameters in mobilenet v3 small: {params_in_small_mobile_net_v3}")
print(f"Number of parameters in mobilenet pytorch v3 small: {params_in_small_mobile_net_v3_pytorch}")
print(f"Ratio: {params_in_small_mobile_net_v3 / params_in_small_mobile_net_v3_pytorch}") # because of bias in conv

Number of parameters in mobilenet v3 small: 2543294
Number of parameters in mobilenet pytorch v3 small: 2542856
Ratio: 1.0001722472684258


### Large mobilenet

In [20]:
large_mobile_net = mobilenet_v3_large(3, 1000)
out = large_mobile_net(x)
assert out.shape == (2, 1000)

In [21]:
from torchvision.models.mobilenetv3 import mobilenet_v3_large as pytorch_mobilenet_v3_large
mobile_net_pytorch = pytorch_mobilenet_v3_large()
params_in_large_mobile_net_v3 = calculate_params(large_mobile_net)
params_in_large_mobile_net_v3_pytorch = calculate_params(mobile_net_pytorch)
print(f"Number of parameters in mobilenet v3 small: {params_in_large_mobile_net_v3}")
print(f"Number of parameters in mobilenet pytorch v3 small: {params_in_large_mobile_net_v3_pytorch}")
print(f"Ratio: {params_in_large_mobile_net_v3 / params_in_large_mobile_net_v3_pytorch}") # because of bias in conv

Number of parameters in mobilenet v3 small: 5493398
Number of parameters in mobilenet pytorch v3 small: 5483032
Ratio: 1.001890559821646


### DeepLab v3

In [22]:
output_stride = 16
out_channels = 256

deeplab_50 = DeepLabv3("resnet50", out_channels=out_channels, num_classes=1)
encoder = deeplab_50.encoder
decoder = deeplab_50.decoder
aspp = decoder.aspp

encoder_out = encoder(x)
assert encoder_out[-1].shape == (2, 2048, h_x // output_stride, w_x // output_stride) # 14x14
assp_out = aspp(encoder_out[-1])
assert assp_out.shape == (2, out_channels, h_x // output_stride, w_x // output_stride)
out = deeplab_50(x)
assert out.shape == (2, 1, h_x, w_x)

print(f"Number of parameters in deeplab v3 resnet50: {calculate_params(deeplab_50)}")

Number of parameters in deeplab v3 resnet50: 25694610


In [23]:
deeplab_101 = DeepLabv3("resnet101", out_channels=out_channels, num_classes=1)
print(f"Number of parameters in deeplab v3 resnet50: {calculate_params(deeplab_101)}")

Number of parameters in deeplab v3 resnet50: 44712850


In [24]:
44712850 / 25694610

1.7401645714801666